In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import csv
import scrapy
import time
import pandas as pd

In [5]:
#you will need to put your own path to the firefox binary here
#binary = FirefoxBinary()
mkeDriver = webdriver.Firefox(executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
legistar = 'https://milwaukee.legistar.com/Legislation.aspx'
mkeDriver.get(legistar)


In [26]:
years = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstYears_Input")

In [30]:
years.send_keys('All Years')

In [31]:
years.submit()

In [32]:
s_button = mkeDriver.find_element_by_id("visibleSearchButton")
s_button.submit()

In [33]:
records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridMain_ctl00")

In [50]:
# for i in records_table.find_elements_by_css_selector('a'):
#     if 'https' in i.get_attribute('href'):
#         print(i.get_attribute('href'))

In [51]:
r_table_body = records_table.find_element_by_css_selector("tbody")

In [ ]:
#can change # of records per page :)

In [56]:
leg_dict = {}
for row in r_table_body.find_elements_by_css_selector('tr'):
    file_num = row.find_element_by_css_selector('a').text
    leg_dict[file_num] = {}
    leg_dict[file_num]['file_link'] = row.find_element_by_css_selector('a').get_attribute('href')
    for d in row.find_elements_by_css_selector('td'):
        leg_dict[file_num]['type'] = d.text
        leg_dict[file_num]['status'] = d.text
        leg_dict[file_num]['created_date'] = d.text
        leg_dict[file_num]['final_action'] = d.text
        leg_dict[file_num]['title'] = d.text
    

In [57]:
#leg_dict

{'181541': {'file_link': 'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=3837397&GUID=F95D0292-D660-42E1-AC11-75FB0A22C3C7&Options=ID|Text|&Search=',
  'type': 'A substitute ordinance relating to traffic controls.',
  'status': 'A substitute ordinance relating to traffic controls.',
  'created_date': 'A substitute ordinance relating to traffic controls.',
  'final_action': 'A substitute ordinance relating to traffic controls.',
  'title': 'A substitute ordinance relating to traffic controls.'},
 '181540': {'file_link': 'https://milwaukee.legistar.com/LegislationDetail.aspx?ID=3837396&GUID=586FBA0E-4766-471F-9D55-ED4B2D04E78F&Options=ID|Text|&Search=',
  'type': 'A substitute ordinance relating to parking controls.',
  'status': 'A substitute ordinance relating to parking controls.',
  'created_date': 'A substitute ordinance relating to parking controls.',
  'final_action': 'A substitute ordinance relating to parking controls.',
  'title': 'A substitute ordinance relating to p